# Exploring Pretraining Datasets

## C4

C4 is a massive dataset used in pre-training T5. In our pre-training setting we only use it as a regularizer so as the model does not forget to understand natural language.

Download link: https://huggingface.co/datasets/allenai/c4/tree/main

In [1]:
import pandas as pd
import json
import random
import gzip
import glob
import numpy as np

from tqdm.notebook import tqdm
from sentence_splitter import split_text_into_sentences
from nltk.tokenize import word_tokenize

### Reading

In [2]:
with open('../storage/datasets/c4/c4_data_original.json') as f:
    c4_original = [json.loads(line)['text'] for line in f]

### Processing

* Removing newlines
* Sentence splitting
* Shuffling

In [3]:
# Remove newlines
c4 = [text.replace('\n', ' ') for text in c4_original]

# Sentence split
c4_sentences = []
for text in tqdm(c4):
    sents = split_text_into_sentences(text, language='en')
    c4_sentences.extend(sents)

random.shuffle(c4_sentences)

del c4

  0%|          | 0/356317 [00:00<?, ?it/s]

## WDC


WebTableCorpus is a collection of tables crawled from the web.

Download link: http://webdatacommons.org/webtables/2015/downloadInstructions.html

## Reading

In [2]:
table_paths = glob.glob('../storage/datasets/wdc/original/1438042981460.12/warc/*')
original_tables = []

def get_tables():
    for table_path in table_paths[:500]:
        with gzip.open(table_path, 'r') as f:
            for line in f:
                try:
                    yield json.loads(line)
                except UnicodeDecodeError:
                    continue
# print(len(original_tables))

## Filtering

In [3]:
def is_english(table):
    return any(domain in table['url'] for domain in ['.com', '.eu', '.uk', '.net', '.org'])

def has_header(table):
    return table['hasHeader']

def is_not_empty(table):
    return len(table['relation']) >= 2

def is_not_huge(table):
    return len(table['relation']) < 50 and len(table['relation'][0]) < 15

def has_title_or_page_title(table):
    return table['title'] != '' or table['pageTitle'] != ''

def title_is_not_huge(table):
    if table['title'] != '' and len(table['title'].split()) < 5:
        return True
    elif table['pageTitle'] != '' and len(table['pageTitle'].split()) < 5:
        return True
    else:
        return False
    

filtered_tables = [table for table in get_tables() 
                  if has_header(table) and 
                  is_not_empty(table) and 
                  is_english(table) and
                  is_not_huge(table) and
                  has_title_or_page_title(table) and
                  title_is_not_huge(table)]

print(len(filtered_tables))

122124


## Analysis of TextBefore and TextAfter

In [6]:
def calculate_token_overlap(text, row):
    text_tokens = word_tokenize(text)
    
    # We want the text to be at least 8 tokens so as it resembles real text
    if len(row) == 0 or len(text_tokens) < 8:
        return 0
    
    token_set = set(text_tokens)
    row_set = set(row)
    
    return len(token_set.intersection(row_set)) / min(len(text_tokens), len(row_set))

def find_overlapping_tokens(text, row):
    text_tokens = word_tokenize(text)

    # We want the text to be at least 8 tokens so as it resembles real text
    if len(row) == 0 or len(text_tokens) < 8:
        return set()

    token_set = set(text_tokens)
    row_set = set(row)

    return token_set.intersection(row_set)

def calculate_row_overlaps(pairs):
    return [len(find_overlapping_tokens(sent, row)) for sent, row in pairs]


def create_possible_sent_row_pairs(table, text_position="textBeforeTable"):
    sent_row_pairs = []
    for sent in table[text_position]:
        for row in table['relation'][1:]:
            sent_row_pairs.append((sent, row))
    # print(sent_row_pairs)
    return sent_row_pairs
            
            
def calculate_all_overlaps(table, text_position="textBeforeTable"):
    if len(table[text_position]) == 0:
        return [0]
    
    pairs = create_possible_sent_row_pairs(table, text_position)
    overlaps = calculate_row_overlaps(pairs)
    return overlaps

    

def calculate_dataset_overlaps(tables):
    before_table_max = []
    after_table_max = []
    
    for table in tqdm(tables):
        
        before_table_max.append(max(calculate_all_overlaps(table, text_position="textBeforeTable")))
        after_table_max.append(max(calculate_all_overlaps(table, text_position="textAfterTable")))
        # break

    return before_table_max, after_table_max


with open('../storage/datasets/wdc/filtered/1438042981525.10.json', 'r') as inp:
    filtered_tables = json.load(inp)

before_overlaps, after_overlaps = calculate_dataset_overlaps(filtered_tables)
# for table in filtered_tables[:10]:
#     print(table)
#     print("-"*90)
    

  0%|          | 0/105119 [00:00<?, ?it/s]

In [8]:
before_overlaps = np.array(before_overlaps)
after_overlaps = np.array(after_overlaps)

## How many tables have at least one row with more than 50% overlap
thresh = 1
print(f"More than {thresh} | TextBefore: {np.sum(before_overlaps >= thresh) / len(before_overlaps)}")
print(f"More than {thresh} | TextAfter: {np.sum(after_overlaps >= thresh) / len(after_overlaps)}")

## What is the average overlap of the text before and the text after?
print(f"Average overlap | TextBefore: {np.mean(before_overlaps)}")
print(f"Average overlap | TextAfter: {np.mean(after_overlaps)}")

More than 1 | TextBefore: 0.07111939801558234
More than 1 | TextAfter: 0.14288568194141876
Average overlap | TextBefore: 0.08692053767634776
Average overlap | TextAfter: 0.26193171548435584


In [14]:
before_overlaps = np.array(before_overlaps)
after_overlaps = np.array(after_overlaps)

thresh = 0.4
after_inds = np.where(after_overlaps > thresh)[0]
before_inds = np.where(before_overlaps > thresh)[0]


for i, ind in enumerate(before_inds):
    table = pd.DataFrame(filtered_tables[ind]['relation'][1:], columns=filtered_tables[ind]['relation'][0])
    print(filtered_tables[ind]['textBeforeTable'])
    display(table)
    
    print("-" * 90)
    
    if i > 20:
        break
    

['Found 233 RPM for libxml_plugin.so RPM resource libxml_plugin.so Arch System The System and Arch are optional added filters, for example System could be "redhat", "redhat-7.2", "mandrake" or "gnome", Arch could be "i386" or "src", etc.', 'depending on your system.', 'The search service can find package by either name (apache), provides(webserver), absolute file names (/usr/bin/apache), binaries (gprof) or shared libraries (libXm.so.2) in standard path.', 'It does not support multiple arguments yet... Help Mirrors index by Name index by creation date index by Vendor index by Distribution index by Group Index']


,Package,videolan-client-0.8.4-3.2.el4.rf.i386.html,videolan-client-0.8.4a-4.el4.rf.i386.html
0,Summary,"The VideoLAN client, also a very good standalo...","The VideoLAN client, also a very good standalo..."
1,Distribution,DAG packages for Red Hat Linux el4 i386,DAG packages for Red Hat Linux el4 i386
2,Download,videolan-client-0.8.4-3.2.el4.rf.i386.rpm,videolan-client-0.8.4a-4.el4.rf.i386.rpm


------------------------------------------------------------------------------------------
['Botswana good excellent excellent excellent excellent excellent excellent good good good good good Matobo poor good excellent excellent excellent excellent good fair fair']


,Moremi,Okavango Delta,Linyanti,Savute,Chobe,Nxai Pan,Makgadikgadi
0,good,good,good,excellent,fair,excellent,excellent
1,good,good,good,excellent,fair,excellent,excellent
2,good,good,good,excellent,fair,excellent,excellent
3,good,good,good,excellent,good,excellent,excellent
4,excellent,excellent,excellent,good,good,good,good
5,excellent,excellent,excellent,good,excellent,fair,fair
6,excellent,excellent,excellent,good,excellent,fair,fair
7,excellent,excellent,excellent,good,excellent,poor,poor
8,excellent,excellent,excellent,good,excellent,poor,poor
9,excellent,excellent,excellent,good,excellent,poor,poor


------------------------------------------------------------------------------------------
['Show/hide PIC10F X=Not Working U = Untested v = Tested ½ = Partially Supported Devices The devices below are supported by the embedded software and the usbpicprog PC software It is very possible that a PIC device is supported by the latest snapshot version of usbpicprog (firmware + PC software).', 'In the table below, either the snapshot revision or the release version of usbpicprog is shown for every supported device.', 'The latest firmware can always be obtained from the github repository in binary and source form (.hex file).', 'The software is available in source format.', 'A ready-built snapshot version can be found here, but be careful – the snapshots are not stable and therefore not yet released.', 'If you have tested a device that is not listed, please use the form below Supported devices usbpicprog a free and open source usb pic programmer']


...

------------------------------------------------------------------------------------------
['Found 3 RPM for techhounds_frc http://www.techhounds.com/ Found 1 sites for techhounds_frc The techhounds-frc package includes the NetBeans IDE, as well as the five plugins necessary to get it to deploy code onto an FIRST Robotics Competion (FRC) robot.', 'It also includes an frc-ping script, which can be used to test network functionality on an FRC robotics network.', 'RPM resource techhounds_frc Arch System The System and Arch are optional added filters, for example System could be "redhat", "redhat-7.2", "mandrake" or "gnome", Arch could be "i386" or "src", etc.', 'depending on your system.', 'The search service can find package by either name (apache), provides(webserver), absolute file names (/usr/bin/apache), binaries (gprof) or shared libraries (libXm.so.2) in standard path.', 'It does not support multiple arguments yet... Help Mirrors index by Name index by creation date index by Vendor

,Package,techhounds_frc-20130207-429.noarch.html,techhounds_frc-20130106-2.noarch.html,techhounds_frc-20130106-2.noarch.html
0,Summary,Software used to work on FRC robotics.,Software used to work on FRC robotics.,Software used to work on FRC robotics.
1,Distribution,SourceForge,SourceForge,SourceForge
2,Download,techhounds_frc-20130207-429.noarch.rpm,techhounds_frc-20130106-2.noarch.rpm,techhounds_frc-20130106-2.noarch.rpm


------------------------------------------------------------------------------------------
['Arkansas Sarah Angeline Burnham Callaway County, Missouri Roda Burnham Callaway County, Missouri Death Certificates Maine Death Records New Hampshire Death Records Kentucky Death Records Illinois Death Records Link to Roots Web message board Death of Adelaide Burnham Of Missouri Death records Search billions of records on Ancestry.com']


,Name,"Burnham, Lydia Reed","Burnham, Rufus"
0,Death Date,"Jan. 19, 1926","July 29, 1936"
1,County,Bradford-O'Keefe Funeral Home,Bradford-O'Keefe Funeral Home


------------------------------------------------------------------------------------------


### Check specific cases

In [31]:
# How many tables contain the word Player in first column
player_tables = 0

for table in filtered_tables:
    if table['relation'][0][0] == 'Cart Icon':
        player_tables += 1
        
player_tables

1238

## Analysis of Title and PageTitle

In [74]:
has_title = 0
has_page_title = 0
has_both = 0
has_none = 0

for table in filtered_tables:
    if table['title'] != '':
        has_title += 1
    if table['pageTitle'] != '':
        has_page_title += 1
    if table['title'] != '' and table['pageTitle'] != '':
        has_both += 1
    if table['title'] == '' and table['pageTitle'] == '':
        has_none += 1
        
print(f"Has title: {has_title / len(filtered_tables)}")
print(f"Has page title: {has_page_title / len(filtered_tables)}")
print(f"Has both: {has_both / len(filtered_tables)}")
print(f"Has none: {has_none / len(filtered_tables)}")

Has title: 0.22697422292096558
Has page title: 0.9997707248370509
Has both: 0.22674494775801643
Has none: 0.0


In [ ]:
# Explore the difference between title and pageTitle
counter = 0
for table in filtered_tables:
    if table['title'] != '':
        print(table)
        counter += 1
        print("-" * 120)
    if counter > 5:
        break
        
print("!" * 240)
        
counter = 0
for table in filtered_tables:
    if table['pageTitle'] != '':
        print(table)
        counter += 1
        print("-" * 120)
    if counter > 5:
        break

### Ratio test

In [4]:
with open('../storage/datasets/pretrain/pretrain_file_0.json', 'r') as inp:
    pretrain_dataset = json.load(inp)
    
def extract_tag(datapoint):
    return datapoint[1:].split('>')[0]

tags = [extract_tag(datapoint['totto_task']) for datapoint in pretrain_dataset]

from collections import Counter

counted_tags = Counter(tags)
counted_tags

Counter({'c4_masking': 1000,
         'wdc_type': 250,
         'wdc_mixing': 250,
         'wdc_content': 250,
         'wdc_masking': 250})